# Intro
This notebook uses the classes bayesopt4ros and contextual_bayesopt4ros but without using ros.


## Bayesian Optimization Example (Not Contextual)

### Util cell
gets the config file and defines some classes to replace the ros2 related parts.

In [55]:
# this notebook uses the classes bayesopt4ros and contextual_bayesopt4ros but without using ros.
# instead of creating a ros2 server
import os
import glob

from bayesopt4ros.bayesopt import BayesianOptimization
from bayesopt4ros import util
import numpy as np
import test_objectives
import torch

# create a class that replaces the logger of ros2 with functions warning, info, debug and error
class Logger:
    def warning(self, msg):
        print(f'WARNING: {msg}')
    def info(self, msg):
        print(f'INFO: {msg}')
    def debug(self, msg):
        print(f'DEBUG: {msg}')
    def error(self, msg):
        print(f'ERROR: {msg}')

#create a class called goal, which has, goal.x_new, and goal.y_new
class Goal:
    def __init__(self, x_new, y_new, c_new= None):
        self.x_new = x_new
        self.y_new = y_new
        self.c_new = c_new

# get the path of the parent of the current directory
current_dir = os.path.dirname(os.path.realpath(''))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

print(f'current_dir: {current_dir}')
print(f'parent_dir: {parent_dir}')
yaml_file = glob.glob(os.path.join('../config', 'forrester_ucb.yaml'))[0]
print(yaml_file)



current_dir: /home/fadi/ros2_ws_git_3/src/external_deps/bayesopt4ros/bayesopt4ros
parent_dir: /home/fadi/ros2_ws_git_3/src/external_deps/bayesopt4ros
../config/forrester_ucb.yaml


In [48]:

N_max = 15
bo = BayesianOptimization.from_file(yaml_file, logger=Logger())
func = test_objectives.Forrester()

# triggering the BO algorithm
goal = Goal(0, 0)
x_new = bo.next(goal)
print(f'x_new: {x_new}')
print('-----------')

# run the BO algorithm
for i in range(N_max):
    x_new_tensor = torch.atleast_2d(x_new.clone().detach())
    # x_new_tensor = torch.atleast_2d(torch.tensor(x_new,dtype=torch.double))
    y_new = func(x_new_tensor).squeeze().item()
    goal = Goal(x_new, y_new)
    x_new = bo.next(goal)
    print(f'iteration {i} :  x_new: {x_new.item():.3f}, y_new: {y_new:.3f}')


INFO: -----------
INFO: feature_names: x1
INFO: outcome_names: y1
x_new: tensor([0.5000], dtype=torch.float64)
-----------
iteration 0 :  x_new: 0.000, y_new: 0.909


[INFO] [1712681586.471141515] [util]: Logging directory: BO_logs/forrester_ucb/2024-04-09-18-53-06
/home/fadi/.local/lib/python3.10/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  stdvs = Y.std(dim=-2, keepdim=True)
/home/fadi/.local/lib/python3.10/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


iteration 1 :  x_new: 1.000, y_new: 3.027
iteration 2 :  x_new: 0.262, y_new: 15.830
iteration 3 :  x_new: 0.363, y_new: -0.140
iteration 4 :  x_new: 0.180, y_new: 0.011
iteration 5 :  x_new: 0.161, y_new: -0.818
iteration 6 :  x_new: 0.678, y_new: -0.939
iteration 7 :  x_new: 0.773, y_new: -3.604
iteration 8 :  x_new: 0.742, y_new: -5.877
iteration 9 :  x_new: 0.757, y_new: -5.898
iteration 10 :  x_new: 0.872, y_new: -6.021
iteration 11 :  x_new: 0.422, y_new: 1.841
iteration 12 :  x_new: 0.503, y_new: 0.247
iteration 13 :  x_new: 0.743, y_new: 0.924
iteration 14 :  x_new: 0.053, y_new: -5.921


The optimal value found by the BO algorithm

In [51]:
x_opt, f_opt = bo.get_optimal_parameters()
x_best, y_best = bo.get_best_observation()

print(f'x_opt = {x_opt}')
print(f'y_opt = {f_opt}')

print(f'x_best = {x_best}')
print(f'y_best = {y_best}')

x_opt = tensor([0.7571], dtype=torch.float64)
y_opt = -6.026422664372911
x_best = tensor([0.7567], dtype=torch.float64)
y_best = -6.020596949238973


## Contextual Bayesian Optimization Example

In [76]:
# this notebook uses the classes bayesopt4ros and contextual_bayesopt4ros but without using ros.
# instead of creating a ros2 server
import os
import glob

from bayesopt4ros.contextual_bayesopt import ContextualBayesianOptimization 
from bayesopt4ros import util
import numpy as np
import warnings

cbo_yaml_file = glob.glob(os.path.join('../config', 'contextual_forrester_ucb.yaml'))[0]
print(cbo_yaml_file)

def sample_context(self) -> np.ndarray:
        """Samples a random context variable to emulate the client."""
        # context_bounds = [b for b in self.func._bounds[self.func.input_dim :]]
        context_bounds = [[0,20]]
        context = torch.tensor([np.random.uniform(b[0], b[1]) for b in context_bounds],dtype=torch.double)
        return context

N_max = 100
cbo = ContextualBayesianOptimization.from_file(cbo_yaml_file, logger=Logger())
func = test_objectives.ContextualForrester()

# triggering the CBO algorithm
c_prev = sample_context(cbo)
goal = Goal(0, 0, c_prev)
x_new = cbo.next(goal)
print(f'x_new: {x_new}')

warnings.filterwarnings("ignore", category=UserWarning)
for i in range(N_max):
        # xc_new = torch.cat((torch.tensor(x_new,dtype=torch.double), c_prev))
        xc_new = torch.cat((x_new.clone().detach(), c_prev))
        xc_new_tensor = torch.atleast_2d(xc_new.clone().detach())
        y_new = func(xc_new_tensor).squeeze().item()

        c_new = sample_context(cbo)
        x_new = cbo.next(Goal(x_new, y_new, c_new))
        c_prev = c_new
        print(f'iteration {i} :  x_new: {x_new.item():.3f}, y_new: {y_new:.3f}, c_new: {c_new.item():.3f}')


../config/contextual_forrester_ucb.yaml
x_new: tensor([0.5000], dtype=torch.float64)
iteration 0 :  x_new: 0.750, y_new: 0.909, c_new: 4.032
iteration 1 :  x_new: 0.250, y_new: -4.985, c_new: 4.200


[INFO] [1712685285.631802184] [util]: Logging directory: BO_logs/contextual_forrester_ucb/2024-04-09-19-54-45


iteration 2 :  x_new: 0.375, y_new: -1.260, c_new: 5.314
iteration 3 :  x_new: 0.117, y_new: -0.634, c_new: 19.195
iteration 4 :  x_new: 1.000, y_new: -8.221, c_new: 2.596
iteration 5 :  x_new: 0.000, y_new: 17.128, c_new: 7.212
iteration 6 :  x_new: 0.726, y_new: -0.579, c_new: 11.842
iteration 7 :  x_new: 0.264, y_new: -2.874, c_new: 13.049
iteration 8 :  x_new: 0.478, y_new: -3.210, c_new: 17.310
iteration 9 :  x_new: 0.000, y_new: 0.363, c_new: 17.539
iteration 10 :  x_new: 0.657, y_new: -5.742, c_new: 0.234
iteration 11 :  x_new: 0.124, y_new: -2.512, c_new: 5.194
iteration 12 :  x_new: 0.094, y_new: -2.882, c_new: 16.136
iteration 13 :  x_new: 0.157, y_new: -7.101, c_new: 12.176
iteration 14 :  x_new: 0.816, y_new: -5.133, c_new: 11.992
iteration 15 :  x_new: 0.162, y_new: -0.128, c_new: 16.198
iteration 16 :  x_new: 0.104, y_new: -6.420, c_new: 12.392
iteration 17 :  x_new: 0.718, y_new: -5.627, c_new: 3.531
iteration 18 :  x_new: 0.083, y_new: -4.543, c_new: 10.731
iteration 19

In [79]:
# testing the learned optimal parameters
results = []
num_test_points = len(func._test_contexts)
# testing on the context points
for context in func._test_contexts:
       results.append(cbo.get_optimal_parameters(context))
print(results)

id = 0
for context, x_opt, f_opt in zip(
            func._test_contexts,
            func._contextual_optimizer,
            func._contextual_optimal_values,
        ):
            result = results[id]
            print(f'context: {context}| x_opt: {x_opt}, actual: {result[0]} | f_opt: {f_opt}, actual: {result[1]}')

            np.testing.assert_almost_equal(result[0], x_opt, decimal=2)
            np.testing.assert_almost_equal(result[1], f_opt, decimal=2)
            id += 1
        
print('All tests passed!')


[(tensor([0.7573], dtype=torch.float64), tensor(-6.0101, dtype=torch.float64)), (tensor([0.7556], dtype=torch.float64), tensor(-5.5072, dtype=torch.float64)), (tensor([0.7537], dtype=torch.float64), tensor(-5.0009, dtype=torch.float64)), (tensor([0.7517], dtype=torch.float64), tensor(-4.4972, dtype=torch.float64)), (tensor([0.7496], dtype=torch.float64), tensor(-3.9966, dtype=torch.float64)), (tensor([0.1153], dtype=torch.float64), tensor(-4.7048, dtype=torch.float64)), (tensor([0.1103], dtype=torch.float64), tensor(-5.4793, dtype=torch.float64)), (tensor([0.1055], dtype=torch.float64), tensor(-6.2647, dtype=torch.float64)), (tensor([0.1008], dtype=torch.float64), tensor(-7.0604, dtype=torch.float64)), (tensor([0.0962], dtype=torch.float64), tensor(-7.8615, dtype=torch.float64)), (tensor([0.0917], dtype=torch.float64), tensor(-8.6582, dtype=torch.float64))]
context: [0.0]| x_opt: [0.757], actual: tensor([0.7573], dtype=torch.float64) | f_opt: -6.021, actual: -6.010144795837524
context: